In [1]:
import tensorflow as tf

In [2]:
data = tf.keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [3]:
data

((array([[[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         ...,
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0

In [4]:
(x_train, y_train), (x_test, y_test) = data

In [5]:
print(y_train, len(y_train), set(y_train))

#出力結果から、y_trainは、データ数60,000、0から9のいずれかのラベルである配列だと分かります。

[9 0 0 ... 3 0 5] 60000 {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}


In [6]:
from tensorflow.keras import utils

#utilsは、主にデータの前処理を行うための便利な関数が含まれています。

In [7]:
#y_trainをOne-hotベクトルに変換
y_train = utils.to_categorical(y_train)
y_train

array([[0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [11]:
#y_testをOne-hotベクトルに変換
y_test = utils.to_categorical(y_test)

In [12]:
#スケーリング
x_train = x_train / 255
x_test = x_test / 255

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense

In [17]:
#Sequential()は、Kerasでニューラルネットワークを作成するためのクラスです。
#この記述によって、Sequentialクラスがインスタンス化され、ニューラルネットワークを作成するための準備が整います。
model = Sequential()

#1つ目の層、つまり入力層を定義するためのもの
#層を追加する時は、毎回addメソッドを使用します。1つ目の層を定義する場合もaddメソッドを利用
#addメソッドの引数に設定されているFlatten()は、追加する層の定義方法の1つです。
#これによって、入力データを1次元データに変換する層を追加できます。
#input_shapeパラメータには、入力データの構造を記述します。
model.add(Flatten(input_shape=(28, 28)))

#Dense()も、層の定義方法の1つで、全結合層を作成するために使用されます。
#結合層とは、1つ前の層（今回の場合は入力層）との間で、双方の全ノード同士がエッジで繋がっている層のことです。
#基本的なニューラルネットワークでは、全結合層がよく利用されます。
#ノード数:公表されている文献などの設定値を参考に、試行錯誤しながら予測性能を向上させることが推奨されています。
model.add(Dense(128, activation='relu'))

#3つ目の層を追加するためのもの
model.add(Dense(10, activation='softmax'))

In [19]:
model.summary()

#サンプル数は基本的に「None」となります。
#これは、ニューラルネットワークへ投入するサンプル数に指定がないことを意味します。
#ステートフルRNNといった特殊なニューラルネットワークの場合、
#入力するサンプル数が決められており、その場合はサンプル数にNoneではなく数値が表示されます。
#Paramsは、その層で特定すべきパラメータ数（重み(エッジの本数)と閾値の数(ノード数)）

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 784)               0         
                                                                 
 dense_6 (Dense)             (None, 128)               100480    
                                                                 
 dense_7 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101770 (397.54 KB)
Trainable params: 101770 (397.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
#Kerasでは、Sequentialクラスのcompileメソッドを利用して、
#lossパラメータで損失関数（モデルが出した【予測値】と、実際の【正解値】がどれくらいズレているのかを計算するための関数）を、optimizerパラメータで最適化方法を指定可能
#任意ですが、metricsパラメータで、評価指標も追加で指定できます。こ

model.compile(loss='categorical_crossentropy',
    optimizer='adam',
    metrics='accuracy')



In [23]:
model.fit(x_train, y_train, batch_size=32, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.5016 - accuracy: 0.8264
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3768 - accuracy: 0.8644
Epoch 3/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3379 - accuracy: 0.8773
Epoch 4/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3143 - accuracy: 0.8851
Epoch 5/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2931 - accuracy: 0.8921


In [28]:
model.evaluate(x_test, y_test)

ValueError: ignored

In [29]:
from tensorflow.keras.preprocessing.image import load_img

In [30]:
img = load_img('/content/unknown_sneaker.png', target_size=(28, 28), color_mode = 'grayscale')
img

In [31]:
from tensorflow.keras.preprocessing.image import img_to_array

In [32]:
array = img_to_array(img)
array

array([[[  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.]],

       [[  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.]],

       [[  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
        [  0.],
    

In [33]:
array.shape

(28, 28, 1)

In [34]:
print(x_train.shape, x_test.shape)

(60000, 28, 28) (10000, 28, 28)


In [ ]:
import numpy as np
array = array.reshape((1, 28, 28))
array.shape

In [35]:
import numpy as np
array = array.reshape((1, 28, 28))
array.shape

(1, 28, 28)

In [36]:
array = array/225

In [37]:
model.predict(array)

1/1 [==============================] - 0s 86ms/step


array([[4.8665978e-09, 1.3965320e-08, 7.3668144e-10, 2.6358503e-08,
        7.9827644e-10, 4.7847334e-05, 1.3780516e-09, 9.9994302e-01,
        5.1424099e-06, 3.8959543e-06]], dtype=float32)